# Analysis of wage and hour case data from US state labor agencies

## Data import and preparation

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [2]:
df = pd.read_csv(
    "input/state_complaints.csv.gz",
    low_memory=False,
    parse_dates=["date_opened", "date_closed", "date_paid"],
)


### Assignment of helper columns

In [3]:
# TODO: add notes
notna_and_nonzero = lambda val: pd.notna(val) and val != 0

df = df.assign(
    # number of days between either paid or closed date and opened date
    case_duration=lambda df: (
        df.apply(
            lambda row: row.date_paid - row.date_opened
            if pd.notna(row.date_paid)
            else row.date_closed - row.date_opened,
            axis=1,
        ).apply(lambda val: val.days)
    ),
)


### Removal of open, dismissed, withdrawn and other cases

Some states provided case statuses that indicated the outcome of the case. Some states provided case statuses that indicated only "closed" or "open" and no more specifics. Others provided no case status at all. In the transform task, I converted the statuses provided into the categories below. Each agency provided definitions of their statuses prior to this, unless otherwise documented. 

Below I will remove case statuses that indicate the case was rejected or withdrawn, or is still open. For states that didn't provide provide statuses, it is my understanding that they only contain closed cases and that open or rejected cases were not provided, so this should make this data as consistent across agencies as possible.

In [4]:
print(df.case_status.unique())


[nan 'closed' 'overturned' 'pending appeal' 'affirmed' 'open' 'dismissed'
 'pending enforcement' 'withdrawn' 'amount exceeds statutory limit']


In [5]:
states_with_status = df.query("case_status.notna()").state_name.unique()
orig_len = len(df)
df = df[
     # from a state that provided a status and has a status indicating
    # the case is concluded
    (
        (df.state_name.isin(states_with_status))
        & (
            df.case_status.isin(
                [
                    "closed",
                    "pending enforcement",
                    "affirmed",
                    "amount exceeds statutory limit",
                    np.NaN,
                ]
            )
        )
    )
    # or from a state that did not provide a status
    | ~(df.state_name.isin(states_with_status))
]
new_len = len(df)
print(f"Removed {orig_len - new_len} rows")
df.fillna("NaN").pipe(
    lambda df: pd.crosstab(index=df.state_name, columns=df.case_status)
)


Removed 29530 rows


case_status,NaN,affirmed,amount exceeds statutory limit,closed,pending enforcement
state_name,,,,,
California,190227,0,0,0,0
Colorado,454,0,0,0,0
Illinois,15,0,0,11126,769
Indiana,17544,0,325,0,1814
Iowa,7370,0,0,0,0
Kansas,10021,0,0,3,0
Kentucky,7137,0,0,0,0
Maine,792,0,0,0,0
Maryland,5051,0,0,0,0


### Multi-violation cases


In [6]:
# TODO: add notes
df = df.drop_duplicates(subset=["case_uuid"], keep="first")


# Analysis

## Amounts

#### Loading separate Texas amount data

The Texas Workforce Commission claimed that the amounts claimed in each case were private financial information and refused to release them in a way that could be associated with the case information. I convinced them to send me a file containing *only* amounts. 

I therefore cannot remove closed cases like I did with the rest of this data, however in the interest of having *some* data from Texas, I will incorporate this to the rest of the amounts. 

**THIS IS SUBJECT TO CHANGE PENDING A RULING FROM THE TEXAS ATTORNEY GENERAL**

In [7]:
tx = (
    pd.read_excel("input/ORR_R005317-081222_from_CBS__C._Hacker__File_date___Amts.xlsx")
    .rename(
        columns={
            "FILG_DT": "filing_year",
            "CLAIMED": "amount_claimed",
            "AWARD_AM": "amount_assessed",
            "PAID": "amount_paid",
        }
    )
    # there are no null values, just 0s
    .query("amount_assessed != 0")
    # because I have no way of excluding closed or rejected cases, I h ave 
    .query("amount_claimed < @df.amount_claimed.quantile(0.99)")
    .assign(state_name="Texas")
)

def append_texas(dataframe):
    """helper function to append texas amount data to dataframe"""
    return pd.concat([dataframe.query("state_name != 'Texas'"), tx])



In [8]:
amt_df = (
    df
    .pipe(append_texas)
    .assign(
        amount_claimed_or_assessed=lambda df: df.apply(
            # use assessed amount because it is the most accurate
            lambda row: row.amount_assessed if pd.notna(row.amount_assessed)
            # if not available, use the amount claimed
            else row.amount_claimed if pd.notna(row.amount_claimed)
            # only use amount paid if neither assessed nor claimed are available
            # because there are so many states taht don't accurately report
            # paid amounts
            else row.amount_paid if pd.notna(row.amount_paid) else np.NaN,
            axis=1,
        )
    )
)


In [9]:
amt_df.amount_claimed_or_assessed.describe()


count    481731.000
mean       3417.354
std       26544.746
min           0.000
25%           0.000
50%         270.000
75%        1500.000
max     5912348.830
Name: amount_claimed_or_assessed, dtype: float64

In [11]:
amt_df.groupby("state_name").amount_claimed_or_assessed.describe()


,count,mean,std,min,25%,50%,75%,max
state_name,,,,,,,,
California,190225.000,2936.615,26163.444,0.000,0.000,0.000,297.500,5912348.830
Colorado,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Illinois,11910.000,4675.315,37815.497,0.000,286.000,960.000,2700.000,1923000.000
Indiana,16956.000,3440.367,36199.890,0.000,292.500,700.000,1730.000,2297378.000
Iowa,7370.000,115.861,625.433,0.000,0.000,0.000,0.000,7625.000
Kansas,9251.000,3731.729,15532.825,0.440,390.085,904.120,2317.720,500000.000
Kentucky,7137.000,3340.072,16041.229,0.000,183.060,374.960,1150.000,125810.820
Maine,792.000,2496.591,8276.580,2.750,222.040,585.750,1551.312,153993.440
Maryland,5051.000,2315.404,7255.993,0.000,352.000,863.310,2120.000,408655.000


## Case duration

In [12]:
cd_df = df.query("case_duration.notna()").copy()
print(cd_df.case_duration.describe())


count   145502.000
mean       232.791
std        369.323
min          0.000
25%         41.000
50%        119.000
75%        268.000
max       4585.000
Name: case_duration, dtype: float64


In [13]:
bins = pd.cut(bins=[0, 7, 28, 180, 365, 730, 999999999], x=cd_df.case_duration)
cd_df.groupby(bins).size().to_frame("count").rename(
    index={
        pd.Interval(0, 7, closed="right"): "0-7 days",
        pd.Interval(7, 28, closed="right"): "7-28 days",
        pd.Interval(28, 180, closed="right"): "28-180 days",
        pd.Interval(180, 365, closed="right"): "180-365 days",
        pd.Interval(365, 730, closed="right"): "365-730 days",
        pd.Interval(730, 999999999, closed="right"): "730+ days",
    }
)


,count
case_duration,
0-7 days,4923
7-28 days,22485
28-180 days,62956
180-365 days,31617
365-730 days,14020
730+ days,9160


In [14]:
cd_df.groupby("state_name").case_duration.describe()


,count,mean,std,min,25%,50%,75%,max
state_name,,,,,,,,
California,14480.000,529.207,305.746,26.000,306.000,446.000,683.000,1985.000
Indiana,3436.000,29.839,28.236,0.000,14.000,24.000,38.000,389.000
Kansas,9105.000,213.001,401.861,0.000,69.000,127.000,208.000,4105.000
South Carolina,14290.000,62.379,54.661,0.000,28.000,53.000,81.000,1203.000
Texas,60683.000,332.816,450.520,29.000,133.000,208.000,323.000,4585.000
Washington,37646.000,72.916,165.037,0.000,18.000,29.000,59.000,1238.000
West Virginia,3770.000,33.124,42.204,0.000,8.000,20.000,40.000,444.000
Wyoming,2092.000,100.001,137.623,0.000,24.000,48.000,124.000,1114.000
